# Multiple sequence alignment

---

## Before Class
In class today we will be implementing a genearlized version of Smith-Waterman algorithm to identify optimal local alignments of multiple.

Prior to class, please do the following:
1. Review slides on sequence alignments in detail
* Focus on how to conceptually translate the algorithm to code

---
Today we will be implementing a generalized form of Smith-Waterman from our previous class. This is a dynamic programming algorithm used for multiple local sequence alignments. For today's class, we have provided the basic implementation of the algorithm but have not populated the functions for 1) scoring the alignment in the or for 2) traceback through the matrix. You will implement these portions of the algorithm in class today.

As a reminder from the slides, the scoring for Smith-Waterman only uses the scores from the positions above, left, and above-left of the current position in the matrix. The main difference today will be dealing with multiple string and the scoring function for the diagonal matrix which is the average of the possible scores given all sequences.

For traceback, you will need to keep track of the direction of the arrows in a matrix and then begin traceback from the maximum value.

---
## Imports

In [1]:
import numpy as np
from itertools import product #iterate bases in two alignments

---
## Implement progressive alignment for multiple sequence alignment

In [2]:
#Define the calultation of diag (substitution) score for alignment of a pair of alignments
def compute_diag_score(aln1_chars,aln2_chars,match,mismatch): #treat mismatch & gap as same
    '''
    Calculate diag score by averaging over all base combinations in alignment 1 and alignment 2
    
    Args:
        aln1_chars (str): bases in alignment 1
        aln2_chars (str): bases in alignment 2
    
    Returns:
        diag_score (float): diag score
    '''
    diag_score = 0
    for aln1_char, aln2_char in product(aln1_chars, aln2_chars):
        if aln1_char == aln2_char:
            diag_score += match
        else:
            diag_score += mismatch
    diag_score /= (len(aln1_chars) * len(aln2_chars))
    return diag_score

In [3]:
#Modify score calculation function from previous class: 
def cal_score(matrix, aln1, aln2, i, j, match, mismatch, gap):
    '''Calculate score for position (i,j) in scoring matrix, also record move to trace back
    
    Args:
        matrix (numpy array): scoring matrix
        i (int): row number
        j (int): column number
        
    Returns:
        score in position (i,j)    
        move to trace back: 0-END, 1-DIAG, 2-UP, 3-LEFT
    Pseudocode:
        aln1_chars = bases of all seqs in alignment 1 in position (i,j)
        aln2_chars = bases of all seqs in alignment 2 in position (i,j)
        calculate scores based on upper-left, up, left neighbors:
        diag_score = compute_diag_score(aln1_chars,aln2_chars)
        up_score = ...
        left_score = ...
        take the maximum:
        score = max(0, diag_score, up_score, left_score)
        move = ...
        
    '''
    score = 0
    move = 0
    
    #Calculate diagonal score for match or mismatch
    aln1_chars = ''.join([seq[i-1] for seq in aln1]) #append all (i-1)th bases in aln1
    aln2_chars = ''.join([seq[j-1] for seq in aln2]) #append all (j-1)th bases in aln2
    diag_score = matrix[i-1][j-1] + compute_diag_score(aln1_chars,aln2_chars, match, mismatch)

    #Calculate from above position
    up_score = matrix[i-1][j] + gap
    
    #Calculate from left position
    left_score = matrix[i][j-1] + gap
    
    #Final score is the max of moves or 0
    score = max(0, diag_score, up_score, left_score)
    
    #move: 0-END, 1-DIAG, 2-UP, 3-LEFT
    move = np.argmax([0, diag_score, up_score, left_score])  #tie cases will be ambiguous

    return score, move

In [4]:
#Hint: use zip()
def traceback(aln1, aln2, traceback_matrix, maximum_position):
    '''Find the opmital path through scoing marix
        
        diagonal: match/mismatch
        up: gap in aln1
        left: gap in aln2
        
    Args:
        score_matrix (numpy array): scoring matrix
        start_row, start_col: starting position (i.e. max_pos) to trace back
        
    Returns:
        aln_final (array of str): results of multiple sequence alignment (e.g. ['GTTGAC','GTT-AC','GTTG-C'])
        
    Pseudocode:
        #Initialize alignment results for aln1 and aln2
        aligned_aln1 = [[] for i in range(len(aln1))]
        aligned_aln2 = [[] for i in range(len(aln2))]
        while current_move != END:
            current_move = traceback_matrix[current_row][current_col]
            if current_move == DIAG:
                for each element 
                ...
            elif current_move == UP:
                ...
            elif current_move == LEFT:
                ...
            
    '''
    #Initialize our parameters
    aligned_aln1 = [[] for i in range(len(aln1))]
    aligned_aln2 = [[] for i in range(len(aln2))]
    aln_final = []
    gap = '-'
    current_row = maximum_position[0]
    current_col = maximum_position[1]
    current_move = None
    
    # We use this instead of the numbers for directions for readability
    END, DIAG, UP, LEFT = range(4)
    
    while current_move != END:
        current_move = traceback_matrix[current_row][current_col]
       
        if current_move == DIAG:
            for aligned_seq, input_seq in zip(aligned_aln1, aln1):
                aligned_seq.append(input_seq[current_row-1])
            for aligned_seq, input_seq in zip(aligned_aln2, aln2):
                aligned_seq.append(input_seq[current_col-1])
            current_row -= 1
            current_col -= 1
        elif current_move == UP:
            for aligned_seq, input_seq in zip(aligned_aln1, aln1):
                aligned_seq.append(input_seq[current_row-1])
            for aligned_seq, input_seq in zip(aligned_aln2, aln2):
                aligned_seq.append(gap)
            current_row -= 1
        elif current_move == LEFT:
            for aligned_seq, input_seq in zip(aligned_aln1, aln1):
                aligned_seq.append(gap)
            for aligned_seq, input_seq in zip(aligned_aln2, aln2):
                aligned_seq.append(input_seq[current_col-1])
            current_col -= 1

    #reverse alignment seqs
    aligned_aln1 = [''.join(aligned_seq[::-1]) for aligned_seq in aligned_aln1]
    aligned_aln2 = [''.join(aligned_seq[::-1]) for aligned_seq in aligned_aln2]
    
    for aligned_seq in aligned_aln1:
        aln_final.append(aligned_seq)
    for aligned_seq in aligned_aln2:
        aln_final.append(aligned_seq)
        
    return aln_final

In [5]:
#Generalize Smith-Waterman algorithm for a pair of alignments
def SmithWaterman_generalized(aln1, aln2, match=3, mismatch=-3, gap=-2):
    '''Smith-Waterman algorithm for local alignment, generalized for a pair of alignments
    
    Args:
        seq1 (array of strs): input alingment 1 (e.g. ['GTTGAC','GTT-AC'])
        seq2 (array of strs): input alingment 2 (e.g. ['GTTGAC','GTT-AC'])
        match: default = 3
        mismatch: default = -3
        gap: default = -2
    
    Returns:
        results of multiple sequence alignment (array of strs) 
        score_matrix (numpy array): scoring matrix
    '''
    
    
    num_rows = len(aln1[0]) +1
    num_cols = len(aln2[0]) +1
    score_matrix = np.zeros(shape=(num_rows,num_cols), dtype=float) #diag scores can be float
    traceback_matrix = np.zeros(shape=(num_rows,num_cols), dtype=int)
    max_score = 0
    max_pos = (0,0)

    #Create scoring matrix
    for i in range(1,num_rows):
        for j in range(1,num_cols): #iteration starts from position (1,1)
            score_matrix[i][j], traceback_matrix[i][j] = cal_score(score_matrix, aln1, aln2, i, j, match, mismatch, gap)
            
            # Keep track of maximum position for trackback
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i,j)
    
    #Traceback the optimal path through scoring matrix
    aln_final = traceback(aln1, aln2, traceback_matrix, max_pos)
    
    return aln_final, score_matrix

In [6]:
aligned_seq1 = 'GTTGAC'
aligned_seq2 = 'GTT-AC'
aln1 = ['GTTGAC', 'GTT-AC']
aln2 = ['AGTTGCG']

In [7]:
SmithWaterman_generalized(aln1,aln2)

(['GTTGAC', 'GTT-AC', 'GTTG-C'],
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  3.,  1.,  0.,  3.,  1.,  3.],
        [ 0.,  0.,  1.,  6.,  4.,  2.,  0.,  1.],
        [ 0.,  0.,  0.,  4.,  9.,  7.,  5.,  3.],
        [ 0.,  0.,  0.,  2.,  7.,  9.,  7.,  5.],
        [ 0.,  3.,  1.,  0.,  5.,  7.,  6.,  4.],
        [ 0.,  1.,  0.,  0.,  3.,  5., 10.,  8.]]))